# proc_raw

This notebook applies various basic processing and conversions to the raw FlowMow2 data.

#### Setup

In [ ]:
%matplotlib notebook
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib as mpl
import matplotlib.pyplot as plt
import subprocess
import os

#### Open raw data store

In [ ]:
store = pd.HDFStore('flowmow.h5')

#### Get UTM northing and easting values

In [ ]:
# use GMT to get UTM zone 9T northing and easting values
scc = store.scc
tmpfile = dt.datetime.now().strftime("utmx_utmy_%Y%m%d%H%M%S%f")
scc.to_csv(tmpfile, sep=',', columns = ['lon', 'lat'], header=False, index=False)
cmd = 'cat %s | gmt mapproject -Ju9/1 -R-132/-126/40/48 -F' % tmpfile
gmt_output = (subprocess.check_output(cmd, shell=True).decode('utf-8')).split('\n')
os.remove(tmpfile)
os.remove('gmt.history')

# loop through GMT output and add to dataframe
x = []
y = []
for i in gmt_output:
    try:
        x.append(np.float64(i.split('\t')[0]))
        y.append(np.float64(i.split('\t')[1]))
    except:
        pass

# add to scc and store
scc['x'] = x
scc['y'] = y
store['scc'] = scc

#### Build a rotated coordinate system

In [ ]:
# rotate by omega degrees
omega = 29
theta = np.deg2rad(omega)
v = np.transpose(np.asarray([x, y]))
R = np.array([[np.cos(theta),-np.sin(theta)], [np.sin(theta),np.cos(theta)]])
vr = v.dot(R)

# add to scc and store
scc['xrot'] = vr[:,0]
scc['yrot'] = vr[:,1]
store['scc'] = scc

#### Show rotated survey lines

In [ ]:
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

# create a new figure
p = figure(tools='pan,wheel_zoom,box_zoom,reset', match_aspect=True, output_backend="webgl", lod_threshold=100)

# add a circle renderer with vecorized colors and sizes
shiftx = -15
shifty = -680
p.circle(x-min(x)+shiftx, y-min(y)+shifty, radius=1, fill_color='blue', fill_alpha=0.6, line_color=None)
p.circle(scc.xrot-scc.xrot.min()+shiftx, scc.yrot-scc.yrot.min()+shifty, radius=1, fill_color='green',
         fill_alpha=0.6, line_color=None)

# show the results
show(p)

#### Convert SBE3 values to temperature

In [ ]:
# load SBE3
sbe3 = store['sbe3']

# ITS-90 equation
def its90(f,g,h,i,j,f0):
    return 1/(g + h*(np.log(f0/f)) + i*((np.log(f0/f))**2) + j*((np.log(f0/f))**3)) - 273.15

# calibration constants
g_2446 = 4.37247477e-3
h_2446 = 6.48296136e-4
i_2446 = 2.37876805e-5
j_2446 = 2.19965081e-6
f0_2446 = 1000.0
g_2265 = 4.33185264e-3
h_2265 = 6.43918821e-4
i_2265 = 2.36253120e-5
j_2265 = 2.24297344e-6
f0_2265 = 1000.0
Fxtal = 4.91548

# frequencies
sbe3['freq_0'] = 512*Fxtal/(sbe3.counts_0/1e6)
sbe3['freq_1'] = 512*Fxtal/(sbe3.counts_1/1e6)

# temperatures
# we know that 2265 was on the stinger and i have high confidence that 2265 is counts_0
# and 2446 is counts_1 because the cals are not interchangeable
sbe3['temp_stinger'] = its90(sbe3['freq_0'],g_2265,h_2265,i_2265,j_2265,f0_2265)
sbe3['temp_top'] = its90(sbe3['freq_1'],g_2446,h_2446,i_2446,j_2446,f0_2446)

In [ ]:
p = figure(tools='pan,wheel_zoom,box_zoom,reset', match_aspect=True, output_backend="webgl",
           lod_threshold=100, y_range=(2.4,3.4))
p.circle(sbe3.timestamp, sbe3.temp_stinger, radius=1, color='blue')
p.circle(sbe3.timestamp, sbe3.temp_top, radius=1, color='green')
# show the results
show(p)

#### Convert Paros values to temperature and pressure

In [ ]:
# calibration constants from raw output (check that these don't change on other dives)
C1 = -28183.59
C2 = 204.9442
C3 = 87027.50
D1 = 0.0351403
D2 = 0.0
T1 = 30.06669
T2 = 1.427388
T3 = 63.14868
T4 = 205.6885
T5 = 0.0
U0 = 5.834068
Y1 = -3858.945
Y2 = -11153.40
Y3 = 0.0

# load data
paros = store.paros

# 
U = paros.eta - U0
C = C1 + C2*U + C3*U**2
D = D1 + D2*U
T0 = T1 + T2*U + T3*U**2 + T4*U**3 + T5*U**4

paros['temp'] = Y1*U + Y2*U**2 + Y3*U**3
paros['pressure'] = C*(1-T0**2/paros.tau**2)*(1-D*(1-T0**2/paros.tau**2))



#paros['temp'] = 

In [ ]:
(paros.pressure*0.7030696).plot(linewidth=0.5)

In [ ]:
paros.info()